In [ ]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork

import networkx as nx
import pandas as pd

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
network: BaseNetwork = factory.new_network(
    'round-robin',
    teams=6,
    days_between_rounds=3,
    seasons=4,
    league_teams=6,
    league_promotion=0,
    create=True
)
network.create_data()

In [ ]:
network.print_data(schedule=True)

In [ ]:
network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.1, 0.1], beta_parameter=0.006),
    forecast_name='bookmaker_forecast',
    base_ranking='true_rating'
)

network.add_forecast(
    forecast=SimpleForecast(outcomes=['home', 'draw', 'away'], probs=[0.4523, 0.2975, 0.2502]),
    forecast_name='home_forecast',
    base_ranking='true_rating'
)

In [ ]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=-0.1)
)
network.add_odds(
    bookmaker_name="bm",
    bookmaker=bookmaker,
    base_forecast='bookmaker_forecast'
)

In [ ]:
betting = FixedBetting(1000)
network.add_bets(
    bettor_name='b',
    bookmaker='bm',
    betting=betting,
    base_forecast='home_forecast'
)

In [ ]:
rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'])
betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'])

In [ ]:
p = {
    "network": network,
    "player": ["b"],
    "bookmaker": ["bm"],
    "forecast": ['home_forecast']
}

In [ ]:
analysis_dict = []
for away, home, match_id, match_attributes in p['network'].iterate_over_games():
    new_row = None
    for f in p['forecast']:
        new_row = new_row or {
            "HomeTeam": home,
            "AwayTeam": away,
            "Season": match_attributes.get('season', None),
            "Round": match_attributes.get('round', None),
            "Result": match_attributes.get('winner', None),
        }
        forecast_object: BaseForecast = match_attributes.get('forecasts', {}).get(f, None)
        if forecast_object is not None:
            for i, outcome in enumerate(forecast_object.outcomes):
                new_row[outcome] = forecast_object.probabilities[i]
        correct, rps_score = rps.eval(forecast_object.probabilities, observed_result=match_attributes.get('winner'))
        new_row["ForecastName"] = f
        new_row['RPS'] = rps_score if correct else None
        #analysis_dict.append(new_row)
    for o_number, o in enumerate(p['bookmaker']):
        new_row = new_row or {
            "HomeTeam": home,
            "AwayTeam": away,
            "Season": match_attributes.get('season', None),
            "Round": match_attributes.get('season', None),
            "Result": match_attributes.get('winner', None),
        }
        forecast_object: BaseForecast = match_attributes.get('forecasts', {}).get('true_forecast', None)
        if forecast_object is not None:
            for i, outcome in enumerate(forecast_object.outcomes):
                new_row[outcome] = forecast_object.probabilities[i]
        match_odds = match_attributes.get('odds', {}).get(o, [])
        match_bets = match_attributes.get('bets', {}).get(p['player'][o_number], [])
        for i_odd, odd in enumerate(match_odds):
            new_row[f"odds#{forecast_object.outcomes[i_odd]}"] = odd
        for i_bet, bet in enumerate(match_bets):
            new_row[f"bet#{forecast_object.outcomes[i_bet]}"] = bet
        new_row['Bettor'] = p['player'][o_number]
        new_row['Bookmaker'] = o
        expected_results, actual_results = betting_returns.eval(
            bets=match_bets,
            bettor_predictions=forecast_object.probabilities,
            bookmaker_odds=match_odds,
            observed_result=new_row['Result']
        )
        for i_bet, bet in enumerate(match_bets):
            new_row[f"expected#bet#{forecast_object.outcomes[i_bet]}"] = expected_results[i_bet]
            new_row[f"return#bet#{forecast_object.outcomes[i_bet]}"] = actual_results[i_bet]
        analysis_dict.append(new_row)

In [16]:
df = pd.DataFrame(analysis_dict)
df.sort_values(by=['Season', 'Round'], inplace=True)

In [17]:
df

,HomeTeam,AwayTeam,Season,Round,Result,home,draw,away,ForecastName,RPS,odds#home,odds#draw,odds#away,bet#home,bet#draw,bet#away,Bettor,Bookmaker,expected#bet#home,return#bet#home,expected#bet#draw,return#bet#draw,expected#bet#away,return#bet#away
0,3,0,0,0,None,0.27202,0.27474,0.45323,home_forecast,0.38339,2.46301,3.90607,4.04741,10.00000,10.00000,10.00000,b,bm,-3.30006,-10.00000,0.73171,-10.00000,8.34419,-10.00000
1,4,1,0,0,None,0.33967,0.28449,0.37584,home_forecast,0.38339,2.09013,4.22378,5.15740,0.00000,10.00000,10.00000,b,bm,-0.00000,-0.00000,2.01613,-10.00000,9.38370,-10.00000
2,5,2,0,0,None,0.02585,0.05305,0.92110,home_forecast,0.38339,20.29610,10.41219,1.30928,10.00000,10.00000,0.00000,b,bm,-4.75419,-10.00000,-4.47603,-10.00000,0.00000,-0.00000
3,0,4,0,1,None,0.04202,0.08203,0.87595,home_forecast,0.38339,12.71051,7.20164,1.44601,10.00000,10.00000,0.00000,b,bm,-4.65863,-10.00000,-4.09254,-10.00000,0.00000,-0.00000
4,1,2,0,1,None,0.18492,0.23786,0.57721,home_forecast,0.38339,3.34488,3.78712,2.88956,10.00000,10.00000,0.00000,b,bm,-3.81454,-10.00000,-0.99180,-10.00000,0.00000,-0.00000
5,3,5,0,1,None,0.29157,0.27901,0.42942,home_forecast,0.38339,2.33750,3.98082,4.34632,10.00000,10.00000,10.00000,b,bm,-3.18461,-10.00000,1.10696,-10.00000,8.66404,-10.00000
6,5,0,0,2,None,0.28006,0.27665,0.44328,home_forecast,0.38339,2.40925,3.93507,4.16844,10.00000,10.00000,10.00000,b,bm,-3.25256,-10.00000,0.88655,-10.00000,8.47787,-10.00000
7,1,3,0,2,None,0.77712,0.14129,0.08159,home_forecast,0.38339,1.24607,13.96513,28.60254,0.00000,10.00000,10.00000,b,bm,-0.00000,-0.00000,9.73149,-10.00000,13.33615,-10.00000
8,2,4,0,2,None,0.39018,0.28363,0.32620,home_forecast,0.38339,1.89603,4.56652,6.14670,0.00000,10.00000,10.00000,b,bm,-0.00000,-0.00000,2.95180,-10.00000,10.05051,-10.00000
9,4,3,0,3,None,0.79108,0.13330,0.07561,home_forecast,0.38339,1.23450,14.97965,30.96791,0.00000,10.00000,10.00000,b,bm,-0.00000,-0.00000,9.96797,-10.00000,13.41637,-10.00000


In [ ]:
analysis_dict